# JetBot Scripts Usage

This notebook demonstrates how to use the repository's helper scripts for camera and basic robot checks.

## Prerequisites
- You are running this inside the repo's dev container (`docker compose up --build dev`).
- To access a USB camera (`/dev/video0`) from this notebook, start the dev container with device mapping: 
  
  `docker compose run --rm --service-ports --device /dev/video0:/dev/video0 dev`
- For CSI (Raspberry Pi-style) camera, ensure Jetson Argus (GStreamer) is available.
- Port 8080 is exposed for viewing the live stream in a browser.

In [ ]:
# System info (optional)
import sys, os, cv2
print('Python:', sys.version.split()[0])
print('OpenCV:', cv2.__version__)
os.system('ls -l /dev/video* 2>/dev/null || true')


## 1) Detect Camera Type
This prints V4L2 devices (USB/UVC), checks CSI (Argus) availability, and suggests a run command.

In [ ]:
!python scripts/detect_camera.py

## 2) Camera Snapshot (Smoke Test)
Saves a single frame to `notebooks/camera_snapshot.jpg` and displays it.

- Override `VIDEO_DEVICE` or `SNAPSHOT_PATH` as needed.

In [ ]:
VIDEO_DEVICE = os.getenv('VIDEO_DEVICE', '/dev/video0')
SNAPSHOT_PATH = os.getenv('SNAPSHOT_PATH', 'notebooks/camera_snapshot.jpg')
print('Using device:', VIDEO_DEVICE)
print('Snapshot path:', SNAPSHOT_PATH)


In [ ]:
# Run the snapshot script
!VIDEO_DEVICE={VIDEO_DEVICE} SNAPSHOT_PATH={SNAPSHOT_PATH} python scripts/test_camera.py

In [ ]:
# Display the saved snapshot
from matplotlib import pyplot as plt, image as mpimg
img = mpimg.imread(SNAPSHOT_PATH)
plt.imshow(img)
plt.axis('off')


## 3) Live Camera Stream (USB/UVC)
Starts an MJPEG HTTP server at `http://<host>:8080`. Interrupt the notebook kernel or run the Stop cell to end it.

In [ ]:
%%bash
set -euo pipefail
nohup python scripts/camera_stream.py --device "${VIDEO_DEVICE:-/dev/video0}" --width 1280 --height 720 --fps 30 --port 8080 > /tmp/camera_stream.log 2>&1 &
echo $! > /tmp/camera_stream.pid
echo "camera_stream.py started with PID $(cat /tmp/camera_stream.pid)"


### Open Stream Link
Opens a link to the stream using this notebook's host/IP.

In [ ]:
from IPython.display import HTML
HTML(
    """<script>
const h = window.location.hostname;
const u = 'http://' + h + ':8080';
document.write('Live stream: <a target=\"_blank\" href=\"'+u+'\" >'+u+'</a>');
</script>
"""
)


## 4) Live Camera Stream (CSI via GStreamer)
For Raspberry Pi-style CSI camera on Jetson. Requires `nvarguscamerasrc`.

In [ ]:
%%bash
set -euo pipefail
nohup python scripts/camera_stream.py --gst --width 1280 --height 720 --fps 30 --port 8080 > /tmp/camera_stream.log 2>&1 &
echo $! > /tmp/camera_stream.pid
echo "camera_stream.py (CSI) started with PID $(cat /tmp/camera_stream.pid)"


### Stop Live Stream

In [ ]:
%%bash
set -euo pipefail
if [[ -f /tmp/camera_stream.pid ]]; then
  PID=$(cat /tmp/camera_stream.pid) || true
  if kill $PID 2>/dev/null; then echo "Stopped camera_stream.py (PID $PID)"; fi
  rm -f /tmp/camera_stream.pid
else
  echo "No running camera_stream.py found"
fi


## 5) JetBot Patrol (Drive Test)
Runs a simple forward/turn loop to validate motors. This requires access to I2C and GPIO on the Jetson, plus the `jetbot` SDK.

Recommended to run from host using Docker Compose hardware profile:

In [ ]:
# Run this from the HOST terminal (not inside the dev container):
# docker compose --profile hardware run --rm jetbot-patrol
#
# Tune via env vars, e.g.:
# LINE_SPEED=0.25 TURN_SPEED=0.2 STEP_SECONDS=1.5 TURN_SECONDS=0.75 PATROL_LAPS=2 \
#   docker compose --profile hardware run --rm jetbot-patrol


---
If you encounter issues, check logs at `/tmp/camera_stream.log`, verify device visibility (`ls -l /dev/video*`), and confirm Argus presence via `gst-inspect-1.0 nvarguscamerasrc`.